### User:

Consider the following code:

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk, ImageDraw, ttk
import pdfkit
import platform


class ImageAnnotationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotation")
        
        # Create the toolbar
        self.toolbar = tk.Frame(self.root, bd=1, relief=tk.RAISED)
        self.toolbar.pack(side=tk.TOP, fill=tk.X)
        
        # Create the buttons in the toolbar
        self.open_btn = tk.Button(self.toolbar, text="Open Image", command=self.open_image)
        self.open_btn.pack(side=tk.LEFT)
        
        self.undo_btn = tk.Button(self.toolbar, text="Undo", command=self.undo_annotation, state=tk.DISABLED)
        self.undo_btn.pack(side=tk.LEFT)
        
        self.redo_btn = tk.Button(self.toolbar, text="Redo", command=self.redo_annotation, state=tk.DISABLED)
        self.redo_btn.pack(side=tk.LEFT)
        
        self.marker_btn = tk.Button(self.toolbar, text="Marker", command=self.activate_marker, relief=tk.RAISED)
        self.marker_btn.pack(side=tk.LEFT)
        
        self.eraser_btn = tk.Button(self.toolbar, text="Eraser", command=self.activate_eraser, relief=tk.RAISED)
        self.eraser_btn.pack(side=tk.LEFT)
        
        self.size_label = tk.Label(self.toolbar, text="Size:")
        self.size_label.pack(side=tk.LEFT)
        
        self.size_scale = tk.Scale(self.toolbar, from_=1, to=10, orient=tk.HORIZONTAL)
        self.size_scale.pack(side=tk.LEFT)
        
        self.lasso_fill_btn = tk.Button(self.toolbar, text="Lasso Fill", command=self.activate_lasso_fill, relief=tk.RAISED)
        self.lasso_fill_btn.pack(side=tk.LEFT)
        
        self.pan_btn = tk.Button(self.toolbar, text="Pan", command=self.activate_pan, relief=tk.RAISED)
        self.pan_btn.pack(side=tk.LEFT)
        
        self.zoom_in_btn = tk.Button(self.toolbar, text="+", command=self.zoom_in)
        self.zoom_in_btn.pack(side=tk.LEFT)
        
        self.zoom_out_btn = tk.Button(self.toolbar, text="-", command=self.zoom_out)
        self.zoom_out_btn.pack(side=tk.LEFT)
        
        self.pan_btn = tk.Button(self.toolbar, text="Pan", command=self.activate_pan, relief=tk.RAISED)
        self.pan_btn.pack(side=tk.LEFT)
        
        self.save_btn = tk.Button(self.toolbar, text="Save", command=self.save_image)
        self.save_btn.pack(side=tk.LEFT)
        
        # Create the canvas for image display and annotation
        self.annotation_canvas = tk.Canvas(self.root)
        self.annotation_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Initialize variables
        self.image_path = None
        self.image = None
        self.annotations = []
        self.current_annotation = None
        self.redo_stack = []
        
        # Overlay coordinate system
        self.overlay_canvas = tk.Canvas(self.annotation_canvas, width=150, height=150, bg="white", highlightthickness=0)
        self.overlay_canvas.place(x=0, y=0, anchor=tk.CENTER)
        self.overlay_canvas.bind("<Configure>", self.update_overlay)
        
        # Bind mouse events to the canvas
        self.annotation_canvas.bind("<Button-1>", self.start_annotation)
        self.annotation_canvas.bind("<B1-Motion>", self.annotate)
        self.annotation_canvas.bind("<ButtonRelease-1>", self.end_annotation)
        
        # Initialize size scale
        self.size_scale.set(5)
        
        # Initialize pan variables
        self.pan_active = False
        self.pan_start_x = 0
        self.pan_start_y = 0
        
        # Initialize zoom variables
        self.zoom_level = 1.0
        self.zoom_center_x = 0
        self.zoom_center_y = 0
        
        # Bind the mouse events to the canvas for panning and zooming:
        self.annotation_canvas.bind("<ButtonPress-1>", self.start_pan)
        self.annotation_canvas.bind("<B1-Motion>", self.pan_image)
        self.annotation_canvas.bind("<ButtonRelease-1>", self.end_pan)

        self.annotation_canvas.bind("<MouseWheel>", self.zoom)
        self.annotation_canvas.bind("<Button-4>", self.zoom)
        self.annotation_canvas.bind("<Button-5>", self.zoom)

        # Bind keyboard shortcuts for undo and redo based on thea platform
        if platform.system() == "Darwin":  # macOS
            self.root.bind("<Command-z>", lambda event: self.undo_annotation())
            self.root.bind("<Command-y>", lambda event: self.redo_annotation())
        else:  # Windows or Linux
            self.root.bind("<Control-z>", lambda event: self.undo_annotation())
            self.root.bind("<Control-y>", lambda event: self.redo_annotation())
                           
    def open_image(self):
        continue_dialog = simpledialog.askstring("Open Image", "Do you want to continue?", parent=self.root)
        if continue_dialog == "continue":
            file_types = [("Image files", "*.jpg;*.jpeg;*.png"), ("All files", "*.*")]
            image_path = filedialog.askopenfilename(filetypes=file_types)
            
            if image_path:
                self.image_path = image_path
                self.load_image()
    
    def load_image(self):
        self.image = Image.open(self.image_path)
        self.display_image()
        
    def display_image(self):
        self.annotation_canvas.delete("all")
        
        image_width, image_height = self.image.size
        canvas_width = self.annotation_canvas.winfo_width()
        canvas_height = self.annotation_canvas.winfo_height()
        
        # Calculate the display size based on the zoom level
        display_width = int(image_width * self.zoom_level)
        display_height = int(image_height * self.zoom_level)
        
        # Resize the image
        resized_image = self.image.resize((display_width, display_height))
        self.display_image_tk = ImageTk.PhotoImage(resized_image)
        
        # Calculate the image position to center it on the canvas
        x = (canvas_width - display_width) // 2
        y = (canvas_height - display_height) // 2
        
        # Display the image on the canvas
        self.annotation_canvas.create_image(x, y, anchor=tk.NW, image=self.display_image_tk)
        
        # Display the annotations
        for annotation in self.annotations:
            self.draw_annotation(annotation)
        
    def start_annotation(self, event):
        if self.current_annotation is not None:
            self.current_annotation["end"] = event.x, event.y
            self.annotations.append(self.current_annotation)
            self.current_annotation = None
            self.undo_btn.config(state=tk.NORMAL)
            self.redo_btn.config(state=tk.DISABLED)
        
       if self.marker_btn["relief"] == tk.SUNKEN:
        color = "red"
    elif self.eraser_btn["relief"] == tk.SUNKEN:
        color = "white"
        self.current_annotation = {"start": (event.x, event.y), "end": None, "color": color, "size": 5}
        self.annotations.append(self.current_annotation)
        self.current_annotation = None
        self.undo_btn.config(state=tk.NORMAL)
        self.redo_btn.config(state=tk.DISABLED)
    elif self.lasso_fill_btn["relief"] == tk.SUNKEN:
        color = "black"
    else:
        color = "blue"
        
        size = self.size_scale.get()
        self.current_annotation = {"start": (event.x, event.y), "end": None, "color": color, "size": size}
        
    def annotate(self, event):
        if self.current_annotation is not None:
            self.current_annotation["end"] = event.x, event.y
            if self.eraser_active:
                # Update eraser annotation with the current mouse position
                self.current_annotation["start"] = (event.x, event.y)
            self.draw_annotation(self.current_annotation)
        
    def end_annotation(self, event):
        if self.current_annotation is not None:
            self.current_annotation["end"] = event.x, event.y
            self.draw_annotation(self.current_annotation)
            self.annotations.append(self.current_annotation)
            self.current_annotation = None
            self.undo_btn.config(state=tk.NORMAL)
            self.redo_btn.config(state=tk.DISABLED)
    
    def draw_annotation(self, annotation):
        start_x, start_y = annotation["start"]
        end_x, end_y = annotation["end"]
        color = annotation["color"]
        size = annotation["size"]
        
        # Apply the zoom level to the coordinates
        start_x = int(start_x * self.zoom_level)
        start_y = int(start_y * self.zoom_level)
        end_x = int(end_x * self.zoom_level)
        end_y = int(end_y * self.zoom_level)
        
        # Calculate the annotation position based on the zoom level
        canvas_width = self.annotation_canvas.winfo_width()
        canvas_height = self.annotation_canvas.winfo_height()
        
        # Calculate the display size based on the zoom level
        display_width = int(canvas_width * self.zoom_level)
        display_height = int(canvas_height * self.zoom_level)
        
        # Calculate the image position to center it on the canvas
        x = (canvas_width - display_width) // 2
        y = (canvas_height - display_height) // 2
        
        # Adjust the coordinates based on the image position
        start_x += x
        start_y += y
        end_x += x
        end_y += y
        
        # Draw the annotation on the canvas
        self.annotation_canvas.create_line(start_x, start_y, end_x, end_y, fill=color, width=size)
    
    def undo_annotation(self):
        if self.annotations:
            annotation = self.annotations.pop()
            self.redo_stack.append(annotation)
            self.annotation_canvas.delete("all")
            self.display_image()
            self.undo_btn.config(state=tk.DISABLED)
            self.redo_btn.config(state=tk.NORMAL)
    
    def redo_annotation(self):
        if self.redo_stack:
            annotation = self.redo_stack.pop()
            self.annotations.append(annotation)
            self.annotation_canvas.delete("all")
            self.display_image()
            self.undo_btn.config(state=tk.NORMAL)
            self.redo_btn.config(state=tk.DISABLED)
    
    def activate_marker(self):
        self.marker_btn.config(relief=tk.SUNKEN)
        self.eraser_btn.config(relief=tk.RAISED)
        self.lasso_fill_btn.config(relief=tk.RAISED)
        self.pan_btn.config(relief=tk.RAISED)
    
    def activate_eraser(self):
        self.marker_btn.config(relief=tk.RAISED)
        self.eraser_btn.config(relief=tk.SUNKEN)
        self.lasso_fill_btn.config(relief=tk.RAISED)
        self.pan_btn.config(relief=tk.RAISED)
        self.eraser_active = True
    
    def activate_lasso_fill(self):
        self.marker_btn.config(relief=tk.RAISED)
        self.eraser_btn.config(relief=tk.RAISED)
        self.lasso_fill_btn.config(relief=tk.SUNKEN)
        self.pan_btn.config(relief=tk.RAISED)
    
    def activate_pan(self):
        self.marker_btn.config(relief=tk.RAISED)
        self.eraser_btn.config(relief=tk.RAISED)
        self.lasso_fill_btn.config(relief=tk.RAISED)
        self.pan_btn.config(relief=tk.SUNKEN)
        
    def start_pan(self, event):
    self.pan_active = True
    self.pan_start_x = event.x
    self.pan_start_y = event.y

    def pan_image(self, event):
        if self.pan_active:
            delta_x = event.x - self.pan_start_x
            delta_y = event.y - self.pan_start_y
            self.annotation_canvas.xview_scroll(-delta_x, "units")
            self.annotation_canvas.yview_scroll(-delta_y, "units")
            self.pan_start_x = event.x
            self.pan_start_y = event.y

    def end_pan(self, event):
    self.pan_active = False
    
    def update_overlay(self, event):
        self.overlay_canvas.delete("all")
        
        width = event.width
        height = event.height
        
        # Draw a crosshair on the overlay canvas
        self.overlay_canvas.create_line(0, height//2, width, height//2, fill="black", width=1)
        self.overlay_canvas.create_line(width//2, 0, width//2, height, fill="black", width=1)
        
    def zoom(self, event):
        if event.delta > 0:
            self.zoom_in()
        else:
            self.zoom_out()
    
    def zoom_in(self):
        self.zoom_level *= 1.1
        self.display_image()
        
    def zoom_out(self):
        self.zoom_level /= 1.1
        self.display_image()
        
    def save_image(self):
        save_path = filedialog.asksaveasfilename(defaultextension=".jpg")
        
        if save_path:
            # Create a copy of the original image
            image_copy = self.image.copy()
            
            # Create a new image with the annotations
            annotated_image = ImageDraw.Draw(image_copy)
            for annotation in self.annotations:
                start_x, start_y = annotation["start"]
                end_x, end_y = annotation["end"]
                color = annotation["color"]
                size = annotation["size"]
                
                # Apply the zoom level to the coordinates
                start_x = int(start_x * self.zoom_level)
                start_y = int(start_y * self.zoom_level)
                end_x = int(end_x * self.zoom_level)
                end_y = int(end_y * self.zoom_level)
                
                annotated_image.line([(start_x, start_y), (end_x, end_y)], fill=color, width=size)
            
            # Save the annotated image
            annotated_image.save(save_path)
            
            messagebox.showinfo("Image Saved", "The annotated image has been saved successfully.")
    
    def run(self):
        self.root.mainloop()


if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAnnotationApp(root)
    app.run()

Notice that while the program has a marker button, it does not function properly. 

I want two separate marker buttons to exist. 

One button, titled 'white marker', creates white annotations atop the image when the user clicks and drags across the screen. 

The other button, titled 'black marker', creates black annotations atop the image when the user clicks and drags across the screen. 

One essential element of the annotations is that none of the annotations can be drawn beyond the edge of the image's border.

### Chat GPT:

To add separate marker buttons for white and black annotations, and prevent annotations from being drawn beyond the image's border, you can make the following modifications to the code:

1. Add two separate marker buttons: one for white annotations and one for black annotations.

2. Modify the start_annotation method to set the color based on the active marker button.

3. Update the annotate method to prevent annotations from being drawn beyond the image's border.

Here's the modified code:

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk, ImageDraw, ttk
import pdfkit
import platform


class ImageAnnotationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotation")
        
        # Create the toolbar
        self.toolbar = tk.Frame(self.root, bd=1, relief=tk.RAISED)
        self.toolbar.pack(side=tk.TOP, fill=tk.X)
        
        # Create the buttons in the toolbar
        self.open_btn = tk.Button(self.toolbar, text="Open Image", command=self.open_image)
        self.open_btn.pack(side=tk.LEFT)
        
        self.undo_btn = tk.Button(self.toolbar, text="Undo", command=self.undo_annotation, state=tk.DISABLED)
        self.undo_btn.pack(side=tk.LEFT)
        
        self.redo_btn = tk.Button(self.toolbar, text="Redo", command=self.redo_annotation, state=tk.DISABLED)
        self.redo_btn.pack(side=tk.LEFT)
        
        self.white_marker_btn = tk.Button(self.toolbar, text="White Marker", command=self.activate_white_marker, relief=tk.RAISED)
        self.white_marker_btn.pack(side=tk.LEFT)
        
        self.black_marker_btn = tk.Button(self.toolbar, text="Black Marker", command=self.activate_black_marker, relief=tk.RAISED)
        self.black_marker_btn.pack(side=tk.LEFT)
        
        self.size_label = tk.Label(self.toolbar, text="Size:")
        self.size_label.pack(side=tk.LEFT)
        
        self.size_scale = tk.Scale(self.toolbar, from_=1, to=10, orient=tk.HORIZONTAL)
        self.size_scale.pack(side=tk.LEFT)
        
        self.lasso_fill_btn = tk.Button(self.toolbar, text="Lasso Fill", command=self.activate_lasso_fill, relief=tk.RAISED)
        self.lasso_fill_btn.pack(side=tk.LEFT)
        
        self.pan_btn = tk.Button(self.toolbar, text="Pan", command=self.activate_pan, relief=tk.RAISED)
        self.pan_btn.pack(side=tk.LEFT)
        
        self.zoom_in_btn = tk.Button(self.toolbar, text="+", command=self.zoom_in)
        self.zoom_in_btn.pack(side=tk.LEFT)
        
        self.zoom_out_btn = tk.Button(self.toolbar, text="-", command=self.zoom_out)
        self.zoom_out_btn.pack(side=tk.LEFT)
        
        self.pan_btn = tk.Button(self.toolbar, text="Pan", command=self.activate_pan, relief=tk.RAISED)
        self.pan_btn.pack(side=tk.LEFT)
        
        self.save_btn = tk.Button(self.toolbar, text="Save", command=self.save_image)
        self.save_btn.pack(side=tk.LEFT)
        
        # Create the canvas for image display and annotation
        self.annotation_canvas = tk.Canvas(self.root)
        self.annotation_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Initialize variables
        self.image_path = None
        self.image = None
        self.annotations = []
        self.current_annotation = None
        self.redo_stack = []
        
        # Overlay coordinate system
        self.overlay_canvas = tk.Canvas(self.annotation_canvas, width=150, height=150, bg="white", highlightthickness=0)
               self.overlay_canvas.place(x=10, y=10)
        
        # Set initial marker button state
        self.active_marker_button = self.white_marker_btn
        
        # Bind mouse events to the canvas
        self.annotation_canvas.bind("<Button-1>", self.start_annotation)
        self.annotation_canvas.bind("<B1-Motion>", self.annotate)
        self.annotation_canvas.bind("<ButtonRelease-1>", self.end_annotation)
        
    def open_image(self):
        self.image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        
        if self.image_path:
            self.image = Image.open(self.image_path)
            self.image = self.image.convert("RGBA")
            
            # Resize the image to fit the canvas
            canvas_width = self.annotation_canvas.winfo_width()
            canvas_height = self.annotation_canvas.winfo_height()
            self.image = self.image.resize((canvas_width, canvas_height), Image.ANTIALIAS)
            
            self.display_image()
            self.clear_annotations()
        
    def display_image(self):
        # Convert the image to Tkinter-compatible format
        self.display_image = ImageTk.PhotoImage(self.image)
        
        # Display the image on the canvas
        self.annotation_canvas.create_image(0, 0, anchor=tk.NW, image=self.display_image)
        
    def clear_annotations(self):
        self.annotations = []
        self.annotation_canvas.delete("all")
        
    def activate_white_marker(self):
        self.active_marker_button.configure(relief=tk.RAISED)
        self.active_marker_button = self.white_marker_btn
        self.active_marker_button.configure(relief=tk.SUNKEN)
        
    def activate_black_marker(self):
        self.active_marker_button.configure(relief=tk.RAISED)
        self.active_marker_button = self.black_marker_btn
        self.active_marker_button.configure(relief=tk.SUNKEN)
        
    def start_annotation(self, event):
        self.current_annotation = []
        self.current_annotation.append(event.x)
        self.current_annotation.append(event.y)
        
    def annotate(self, event):
        if self.current_annotation:
            x = event.x
            y = event.y
            
            # Prevent annotation from being drawn beyond the image's border
            canvas_width = self.annotation_canvas.winfo_width()
            canvas_height = self.annotation_canvas.winfo_height()
            if x < 0:
                x = 0
            elif x >= canvas_width:
                x = canvas_width - 1
            if y < 0:
                y = 0
            elif y >= canvas_height:
                y = canvas_height - 1
            
            self.current_annotation.append(x)
            self.current_annotation.append(y)
            
            # Draw the annotation on the canvas
            color = self.active_marker_button["text"].lower()
            size = self.size_scale.get()
            self.annotation_canvas.create_line(self.current_annotation, fill=color, width=size, smooth=True)
        
    def end_annotation(self, event):
        if self.current_annotation:
            self.annotations.append(tuple(self.current_annotation))
            self.current_annotation = None
            self.enable_undo_redo_buttons()
        
    def enable_undo_redo_buttons(self):
        self.undo_btn.configure(state=tk.NORMAL)
        self.redo_btn.configure(state=tk.DISABLED)
        
    def undo_annotation(self):
        if self.annotations:
            self.redo_stack.append(self.annotations.pop())
            self.annotation_canvas.delete("all")
            self.display_image()
            
            # Redraw the remaining annotations
            for annotation in self.annotations:
                color = self.get_annotation_color(annotation)
                size = self.get_annotation_size(annotation)
                self.annotation_canvas.create_line(annotation, fill=color, width=size, smooth=True)
            
            if not self.annotations:
                self.disable_undo_redo_buttons()
        
    def redo_annotation(self):
        if self.redo_stack:
            annotation = self.redo_stack.pop()
            self.annotations.append(annotation)
            color = self.get_annotation_color(annotation)
            size = self.get_annotation_size(annotation)
            self.annotation_canvas.create_line(annotation, fill=color, width=size, smooth=True)
            
            self.enable_undo_redo_buttons()
            
    def disable_undo_redo_buttons(self):
        self.undo_btn.configure(state=tk.DISABLED)
        self.redo_btn.configure(state=tk.DISABLED)
        
    def get_annotation_color(self, annotation):
        x1, y1, x2, y2 = annotation
        mid_x = (x1 + x2) // 2
        mid_y = (y1 + y2) // 2
        return self.annotation_canvas.gettags(self.annotation_canvas.find_closest(mid_x, mid_y))[0]
    
    def get_annotation_size(self, annotation):
        x1, y1, x2, y2 = annotation
        return abs(x2 - x1)
    
    def save_annotations(self):
        # Create a new image with annotations
        annotated_image = self.image.copy()
        draw = ImageDraw.Draw(annotated_image)
        
        for annotation in self.annotations:
            x1, y1, x2, y2 = annotation
            color = self.get_annotation_color(annotation)
            draw.line([(x1, y1), (x2, y2)], fill=color, width=self.get_annotation_size(annotation))
        
        # Save the annotated image
        save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG", "*.png")])
        
        if save_path:
            annotated_image.save(save_path)
            messagebox.showinfo("Saved", "Annotations saved successfully.")
        
    def run(self):
        self.window.mainloop()
        
if __name__ == "__main__":
    app = AnnotationApp()
    app.run()

Please note that this code assumes you have the necessary Tkinter and Pillow libraries installed. 

You may need to adjust the code if you are using a different GUI toolkit or image processing library.

### User:

Consider the following code:

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk, ImageDraw, ttk
import pdfkit
import platform


class ImageAnnotationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotation")
        
        # Create the toolbar
        self.toolbar = tk.Frame(self.root, bd=1, relief=tk.RAISED)
        self.toolbar.pack(side=tk.TOP, fill=tk.X)
        
        # Create the buttons in the toolbar
        self.open_btn = tk.Button(self.toolbar, text="Open Image", command=self.open_image)
        self.open_btn.pack(side=tk.LEFT)
        
        self.undo_btn = tk.Button(self.toolbar, text="Undo", command=self.undo_annotation, state=tk.DISABLED)
        self.undo_btn.pack(side=tk.LEFT)
        
        self.redo_btn = tk.Button(self.toolbar, text="Redo", command=self.redo_annotation, state=tk.DISABLED)
        self.redo_btn.pack(side=tk.LEFT)
        
        self.white_marker_btn = tk.Button(self.toolbar, text="White Marker", command=self.activate_white_marker, relief=tk.RAISED)
        self.white_marker_btn.pack(side=tk.LEFT)
        
        self.black_marker_btn = tk.Button(self.toolbar, text="Black Marker", command=self.activate_black_marker, relief=tk.RAISED)
        self.black_marker_btn.pack(side=tk.LEFT)
        
        self.eraser_btn = tk.Button(self.toolbar, text="Eraser", command=self.activate_eraser, relief=tk.RAISED)
        self.eraser_btn.pack(side=tk.LEFT)
        
        self.size_label = tk.Label(self.toolbar, text="Size:")
        self.size_label.pack(side=tk.LEFT)
        
        self.size_scale = tk.Scale(self.toolbar, from_=1, to=10, orient=tk.HORIZONTAL)
        self.size_scale.pack(side=tk.LEFT)
        
        self.lasso_fill_btn = tk.Button(self.toolbar, text="Lasso Fill", command=self.activate_lasso_fill, relief=tk.RAISED)
        self.lasso_fill_btn.pack(side=tk.LEFT)
        
        self.pan_btn = tk.Button(self.toolbar, text="Pan", command=self.activate_pan, relief=tk.RAISED)
        self.pan_btn.pack(side=tk.LEFT)
        
        self.zoom_in_btn = tk.Button(self.toolbar, text="+", command=self.zoom_in)
        self.zoom_in_btn.pack(side=tk.LEFT)
        
        self.zoom_out_btn = tk.Button(self.toolbar, text="-", command=self.zoom_out)
        self.zoom_out_btn.pack(side=tk.LEFT)
        
        self.save_btn = tk.Button(self.toolbar, text="Save", command=self.save_image)
        self.save_btn.pack(side=tk.LEFT)
        
        # Create the canvas for image display and annotation
        self.annotation_canvas = tk.Canvas(self.root)
        self.annotation_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Initialize variables
        self.image_path = None
        self.image = None
        self.annotations = []
        self.current_annotation = None
        self.redo_stack = []
        
        # Overlay coordinate system
        self.overlay_canvas = tk.Canvas(self.annotation_canvas, width=150, height=150, bg="white", highlightthickness=0)
        self.overlay_canvas.place(x=0, y=0, anchor=tk.CENTER)
        self.overlay_canvas.bind("<Configure>", self.update_overlay)
        
        # Bind mouse events to the canvas
        self.annotation_canvas.bind("<Button-1>", self.start_annotation)
        self.annotation_canvas.bind("<B1-Motion>", self.annotate)
        self.annotation_canvas.bind("<ButtonRelease-1>", self.end_annotation)
        
        # Initialize size scale
        self.size_scale.set(5)
        
        # Initialize pan variables
        self.pan_active = False
        self.pan_start_x = 0
        self.pan_start_y = 0
        
        # Initialize zoom variables
        self.zoom_level = 1.0
        self.zoom_center_x = 0
        self.zoom_center_y = 0
        
        # Set initial marker button state
        self.active_marker_button = self.white_marker_btn
        
        # Bind the mouse events to the canvas for panning and zooming:
        self.annotation_canvas.bind("<ButtonPress-1>", self.start_pan)
        self.annotation_canvas.bind("<B1-Motion>", self.pan_image)
        self.annotation_canvas.bind("<ButtonRelease-1>", self.end_pan)

        self.annotation_canvas.bind("<MouseWheel>", self.zoom)
        self.annotation_canvas.bind("<Button-4>", self.zoom)
        self.annotation_canvas.bind("<Button-5>", self.zoom)

        # Bind keyboard shortcuts for undo and redo based on thea platform
        if platform.system() == "Darwin":  # macOS
            self.root.bind("<Command-z>", lambda event: self.undo_annotation())
            self.root.bind("<Command-y>", lambda event: self.redo_annotation())
        else:  # Windows or Linux
            self.root.bind("<Control-z>", lambda event: self.undo_annotation())
            self.root.bind("<Control-y>", lambda event: self.redo_annotation())
                           
    def open_image(self):
        continue_dialog = simpledialog.askstring("Open Image", "Do you want to continue?", parent=self.root)
        if continue_dialog == "continue":
            file_types = [("Image files", "*.jpg;*.jpeg;*.png"), ("All files", "*.*")]
            image_path = filedialog.askopenfilename(filetypes=file_types)
            
            if image_path:
                self.image_path = image_path
                self.load_image()
    
    def load_image(self):
        self.image = Image.open(self.image_path)
        self.display_image()
        
    def display_image(self):
        self.annotation_canvas.delete("all")
        
        image_width, image_height = self.image.size
        canvas_width = self.annotation_canvas.winfo_width()
        canvas_height = self.annotation_canvas.winfo_height()
        
        # Calculate the display size based on the zoom level
        display_width = int(image_width * self.zoom_level)
        display_height = int(image_height * self.zoom_level)
        
        # Resize the image
        resized_image = self.image.resize((display_width, display_height))
        self.display_image_tk = ImageTk.PhotoImage(resized_image)
        
        # Calculate the image position to center it on the canvas
        x = (canvas_width - display_width) // 2
        y = (canvas_height - display_height) // 2
        
        # Display the image on the canvas
        self.annotation_canvas.create_image(x, y, anchor=tk.NW, image=self.display_image_tk)
        
        # Display the annotations
        for annotation in self.annotations:
            self.draw_annotation(annotation)
        
    def activate_white_marker(self):
        self.active_marker_button.configure(relief=tk.RAISED)
        self.active_marker_button = self.white_marker_btn
        self.active_marker_button.configure(relief=tk.SUNKEN)
        
    def activate_black_marker(self):
        self.active_marker_button.configure(relief=tk.RAISED)
        self.active_marker_button = self.black_marker_btn
        self.active_marker_button.configure(relief=tk.SUNKEN)
    
    def start_annotation(self, event):
        self.current_annotation = []
        self.current_annotation.append(event.x)
        self.current_annotation.append(event.y)
        
    def annotate(self, event):
        if self.current_annotation:
            x = event.x
            y = event.y
            
            # Prevent annotation from being drawn beyond the image's border
            canvas_width = self.annotation_canvas.winfo_width()
            canvas_height = self.annotation_canvas.winfo_height()
            if x < 0:
                x = 0
            elif x >= canvas_width:
                x = canvas_width - 1
            if y < 0:
                y = 0
            elif y >= canvas_height:
                y = canvas_height - 1
            
            self.current_annotation.append(x)
            self.current_annotation.append(y)
            
            # Draw the annotation on the canvas
            color = self.active_marker_button["text"].lower()
            size = self.size_scale.get()
            self.annotation_canvas.create_line(self.current_annotation, fill=color, width=size, smooth=True)
        
    def end_annotation(self, event):
        if self.current_annotation:
            self.annotations.append(tuple(self.current_annotation))
            self.current_annotation = None
            self.enable_undo_redo_buttons()
            
    def draw_annotation(self, annotation):
        start_x, start_y = annotation["start"]
        end_x, end_y = annotation["end"]
        color = annotation["color"]
        size = annotation["size"]
        
        # Apply the zoom level to the coordinates
        start_x = int(start_x * self.zoom_level)
        start_y = int(start_y * self.zoom_level)
        end_x = int(end_x * self.zoom_level)
        end_y = int(end_y * self.zoom_level)
        
        # Calculate the annotation position based on the zoom level
        canvas_width = self.annotation_canvas.winfo_width()
        canvas_height = self.annotation_canvas.winfo_height()
        
        # Calculate the display size based on the zoom level
        display_width = int(canvas_width * self.zoom_level)
        display_height = int(canvas_height * self.zoom_level)
        
        # Calculate the image position to center it on the canvas
        x = (canvas_width - display_width) // 2
        y = (canvas_height - display_height) // 2
        
        # Adjust the coordinates based on the image position
        start_x += x
        start_y += y
        end_x += x
        end_y += y
        
        # Draw the annotation on the canvas
        self.annotation_canvas.create_line(start_x, start_y, end_x, end_y, fill=color, width=size)
    
    def enable_undo_redo_buttons(self):
        self.undo_btn.configure(state=tk.NORMAL)
        self.redo_btn.configure(state=tk.DISABLED)
    
    def undo_annotation(self):
        if self.annotations:
            self.redo_stack.append(self.annotations.pop())
            self.annotation_canvas.delete("all")
            self.display_image()
            
            # Redraw the remaining annotations
            for annotation in self.annotations:
                color = self.get_annotation_color(annotation)
                size = self.get_annotation_size(annotation)
                self.annotation_canvas.create_line(annotation, fill=color, width=size, smooth=True)
            
            if not self.annotations:
                self.disable_undo_redo_buttons()
    
    def redo_annotation(self):
        if self.redo_stack:
            annotation = self.redo_stack.pop()
            self.annotations.append(annotation)
            color = self.get_annotation_color(annotation)
            size = self.get_annotation_size(annotation)
            self.annotation_canvas.create_line(annotation, fill=color, width=size, smooth=True)
            
            self.enable_undo_redo_buttons()
            
    def disable_undo_redo_buttons(self):
        self.unso_btn.configure(state=tk.DISABLED)
        self.redo_btn.configure(state=tk.DISABLED)
    
    def activate_marker(self):
        self.marker_btn.config(relief=tk.SUNKEN)
        self.eraser_btn.config(relief=tk.RAISED)
        self.lasso_fill_btn.config(relief=tk.RAISED)
        self.pan_btn.config(relief=tk.RAISED)
        
    def get_annotation_color(self, annotation):
        x1, y1, x2, y2 = annotation
        mid_x = (x1 + x2) // 2
        mid_y = (y1 + y2) // 2
        return self.annotation_canvas.gettags(self.annotation_canvas.find_closest(mid_x, mid_y))[0]
    
    def get_annotation_size(self, annotation):
        x1, y1, x2, y2 = annotation
        return abs(x2 - x1)
    
    def activate_eraser(self):
        self.marker_btn.config(relief=tk.RAISED)
        self.eraser_btn.config(relief=tk.SUNKEN)
        self.lasso_fill_btn.config(relief=tk.RAISED)
        self.pan_btn.config(relief=tk.RAISED)
        self.eraser_active = True
    
    def activate_lasso_fill(self):
        self.marker_btn.config(relief=tk.RAISED)
        self.eraser_btn.config(relief=tk.RAISED)
        self.lasso_fill_btn.config(relief=tk.SUNKEN)
        self.pan_btn.config(relief=tk.RAISED)
    
    def activate_pan(self):
        self.marker_btn.config(relief=tk.RAISED)
        self.eraser_btn.config(relief=tk.RAISED)
        self.lasso_fill_btn.config(relief=tk.RAISED)
        self.pan_btn.config(relief=tk.SUNKEN)
        
    def start_pan(self, event):
    self.pan_active = True
    self.pan_start_x = event.x
    self.pan_start_y = event.y

    def pan_image(self, event):
        if self.pan_active:
            delta_x = event.x - self.pan_start_x
            delta_y = event.y - self.pan_start_y
            self.annotation_canvas.xview_scroll(-delta_x, "units")
            self.annotation_canvas.yview_scroll(-delta_y, "units")
            self.pan_start_x = event.x
            self.pan_start_y = event.y

    def end_pan(self, event):
    self.pan_active = False
    
    def update_overlay(self, event):
        self.overlay_canvas.delete("all")
        
        width = event.width
        height = event.height
        
        # Draw a crosshair on the overlay canvas
        self.overlay_canvas.create_line(0, height//2, width, height//2, fill="black", width=1)
        self.overlay_canvas.create_line(width//2, 0, width//2, height, fill="black", width=1)
        
    def zoom(self, event):
        if event.delta > 0:
            self.zoom_in()
        else:
            self.zoom_out()
    
    def zoom_in(self):
        self.zoom_level *= 1.1
        self.display_image()
        
    def zoom_out(self):
        self.zoom_level /= 1.1
        self.display_image()
        
    def save_image(self):
        save_path = filedialog.asksaveasfilename(defaultextension=".jpg")
        
        if save_path:
            # Create a copy of the original image
            image_copy = self.image.copy()
            
            # Create a new image with the annotations
            annotated_image = ImageDraw.Draw(image_copy)
            for annotation in self.annotations:
                start_x, start_y = annotation["start"]
                end_x, end_y = annotation["end"]
                color = annotation["color"]
                size = annotation["size"]
                
                # Apply the zoom level to the coordinates
                start_x = int(start_x * self.zoom_level)
                start_y = int(start_y * self.zoom_level)
                end_x = int(end_x * self.zoom_level)
                end_y = int(end_y * self.zoom_level)
                
                annotated_image.line([(start_x, start_y), (end_x, end_y)], fill=color, width=size)
            
            # Save the annotated image
            annotated_image.save(save_path)
            
            messagebox.showinfo("Image Saved", "The annotated image has been saved successfully.")
    
    def run(self):
        self.root.mainloop()


if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAnnotationApp(root)
    app.run()

Notice that the code includes a sliding scale for changing the size of the selected tool. 

Instead, could you modify the code to include a size adjuster for each tool. 

More specifically, include a size slide bar under the black marker button, white marker 
button, and eraser button that adjust the sizing of the tool under which the slider is located.

### Chat GPT:

Certainly!

I have modified the code to include a size slider for each tool. 

Each tool now has its own size scale widget that allows you to adjust the size of the tool's drawing or erasing feature. 

Here's the updated code:

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk, ImageDraw, ttk
import pdfkit
import platform


class ImageAnnotationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotation")
        
        # Create the toolbar
        self.toolbar = tk.Frame(self.root, bd=1, relief=tk.RAISED)
        self.toolbar.pack(side=tk.TOP, fill=tk.X)
        
        # Create the buttons in the toolbar
        self.open_btn = tk.Button(self.toolbar, text="Open Image", command=self.open_image)
        self.open_btn.pack(side=tk.LEFT)
        
        self.undo_btn = tk.Button(self.toolbar, text="Undo", command=self.undo_annotation, state=tk.DISABLED)
        self.undo_btn.pack(side=tk.LEFT)
        
        self.redo_btn = tk.Button(self.toolbar, text="Redo", command=self.redo_annotation, state=tk.DISABLED)
        self.redo_btn.pack(side=tk.LEFT)
        
        self.white_marker_btn = tk.Button(self.toolbar, text="White Marker", command=self.activate_white_marker, relief=tk.RAISED)
        self.white_marker_btn.pack(side=tk.LEFT)
        self.white_marker_size_scale = tk.Scale(self.toolbar, from_=1, to=10, orient=tk.HORIZONTAL)
        self.white_marker_size_scale.pack(side=tk.LEFT)
        
        self.black_marker_btn = tk.Button(self.toolbar, text="Black Marker", command=self.activate_black_marker, relief=tk.RAISED)
        self.black_marker_btn.pack(side=tk.LEFT)
        self.black_marker_size_scale = tk.Scale(self.toolbar, from_=1, to=10, orient=tk.HORIZONTAL)
        self.black_marker_size_scale.pack(side=tk.LEFT)
        
        self.eraser_btn = tk.Button(self.toolbar, text="Eraser", command=self.activate_eraser, relief=tk.RAISED)
        self.eraser_btn.pack(side=tk.LEFT)
        self.eraser_size_scale = tk.Scale(self.toolbar, from_=1, to=10, orient=tk.HORIZONTAL)
        self.eraser_size_scale.pack(side=tk.LEFT)
        
        self.lasso_fill_btn = tk.Button(self.toolbar, text="Lasso Fill", command=self.activate_lasso_fill, relief=tk.RAISED)
        self.lasso_fill_btn.pack(side=tk.LEFT)
        self.lasso_fill_size_scale = tk.Scale(self.toolbar, from_=1, to=10, orient=tk.HORIZONTAL)
        self.lasso_fill_size_scale.pack(side=tk.LEFT)
        
        self.pan_btn = tk.Button(self.toolbar, text="Pan", command=self.activate_pan, relief=tk.RAISED)
        self.pan_btn.pack(side=tk.LEFT)
        
        self.zoom_in_btn = tk.Button(self.toolbar, text="+", command=self.zoom_in)
        self.zoom_in_btn.pack(side=tk.LEFT)
        
        self.zoom_out_btn = tk.Button(self.toolbar, text="-", command=self.zoom_out)
        self.zoom_out_btn.pack(side=tk.LEFT)
        
        self.save_btn = tk.Button(self.toolbar, text="Save", command=self.save_image)
        self.save_btn.pack(side=tk.LEFT)
        
        # Create the canvas for image display and annotation
        self.annotation_canvas = tk.Canvas(self.root)
        self.annotation_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Initialize variables
        self.image_path = None
        self.image = None
        self.annotations = []
        self.